In [2]:
import pandas as pd  #Imports for converting dataframe to train_dev splits
import numpy as np
from pathlib import Path
import nltk, spacy
from nltk.corpus import stopwords
from wordcloud import STOPWORDS
from collections import Counter
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.model_selection import train_test_split 

pd.options.display.float_format = '{:,.0f}'.format

In [3]:
in_file = Path.cwd().parents[0] / 'Processed_datasets' / 'tweets_long_no_duplicates.csv'
my_stopwords = stopwords.words('english')
stopwords = set(STOPWORDS).union(my_stopwords) #preparing stopwards list
custom_stopwords = ['hi', '\n', '\n\n', '&amp;', ' ', '.', '-',
                    'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']
nlp = spacy.load('en_core_web_sm', exclude=['ner', 'tok2vec', 'tagger', 'paerser', 'senter', 'lemmatizer', 'attribute_ruler']) # using only for stopwords
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)
CHUNK_SIZE = 1000

In [4]:
df_long = pd.read_csv(in_file, engine='python', usecols=['Tokens', 'Label'], encoding='utf-8') #not using unnecessary columns

In [33]:
print(df_long.head(15))
df_long.count()
print(df_long['Label'].value_counts())

                                               Tokens  Label
0   muslim mob violence against hindus in banglade...      1
1   islamophobia is like the idea of naziphobia is...      1
2   finally all caught up and that sudden death co...      0
3   please please start using is your discernment ...      0
4   as soon as isis chased all the minorities out ...      0
5   islam invaded and conquered of christiandom be...      1
6   do you approve of your pedophile prophet rapin...      1
7   problem with vile muslims is that they try to ...      1
8            tend to talk about it much personal info      0
9   cool next time when a woman talks to him about...      0
10  our judges are about to turn the heat up in th...      0
11  lol this you walk by putting one foot in front...      0
12  said wanted sorbet now and they should tell us...      0
13  this fucking potato is blowing my mind duck fa...      0
14  omg this churner feels like razor blades on my...      0
0    15214
1     7500
Na

In [5]:
#setting up baseline pipeline
from nltk.tokenize import TweetTokenizer #I chose to tokenize with this, as it gets rid of @ handlers
from sklearn.feature_extraction.text import TfidfVectorizer #easy idf and stopword removal
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler #moderately imbalanced dataset so randomundersampling - approx 2:1 ratio
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline

In [18]:


PARAM_GRID = [{
        'vect__stop_words': [None, ALL_STOP_WORDS],
        'clf__kernel' : ['sigmoid', 'rbf'],
        #'clf__loss' : ['hinge', 'squared_hinge'],
        'vect__use_idf' : [True, False]
    }]


def make_pipeline(df, clf, params, standardise=True):
    if standardise == True:
        xs, ys = df['Tokens'].values, df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(
            xs, ys, train_size=.85, random_state=42, stratify=ys)
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = TfidfVectorizer(
            max_df=.9, min_df=25, strip_accents='unicode', tokenizer=tknzr.tokenize)
        scaler = StandardScaler(with_mean=False)
        sampler = RandomUnderSampler(random_state=42)
        pipeline = Pipeline([('vect', vect), ('scale', scaler),
                             ('sampler', sampler), ('clf', clf)])
        grid_srch = GridSearchCV(
            estimator=pipeline, param_grid=params, refit=True, n_jobs=-1)
        grid_srch.fit(x_train, y_train)  # fit the grid_search object
        prediction = grid_srch.predict(x_dev)  # Obtain predictions and save them
        # obtain classification report of preds
        report = classification_report(y_dev, prediction, output_dict=True)
        best_est = grid_srch.best_estimator_
        print(report)
        return best_est, prediction
    else:
        xs, ys = df['Tokens'].values, df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(
            xs, ys, train_size=.85, random_state=42, stratify=ys)
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = TfidfVectorizer(
            max_df=.9, min_df=25, strip_accents='unicode', tokenizer=tknzr.tokenize)
        scaler = StandardScaler(with_mean=False)
        sampler = RandomUnderSampler(random_state=42)
        pipeline = Pipeline([('vect', vect), ('scale', scaler),
                             ('sampler', sampler), ('clf', clf)])
        grid_srch = GridSearchCV(
            estimator=pipeline, param_grid=params, refit=True, n_jobs=-1)
        grid_srch.fit(x_train, y_train)  # fit the grid_search object
        prediction = grid_srch.predict(x_dev)  # Obtain predictions and save them
        # obtain classification report of preds
        report = classification_report(y_dev, prediction, output_dict=True)
        best_est = grid_srch.best_estimator_
        print(report)
        return best_est, prediction


In [17]:
tup_linear_svc = make_pipeline(df_long, LinearSVC(random_state=42), PARAM_GRID)

C:\Users\amaan\anaconda3\envs\t_mining\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['&', "'", 'n', '‘', '’'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'0': {'precision': 0.8294172932330827, 'recall': 0.7731055628558914, 'f1-score': 0.8002720471548402, 'support': 2283}, '1': {'precision': 0.5953125, 'recall': 0.6773333333333333, 'f1-score': 0.6336798336798337, 'support': 1125}, 'accuracy': 0.7414906103286385, 'macro avg': {'precision': 0.7123648966165413, 'recall': 0.7252194480946124, 'f1-score': 0.716975940417337, 'support': 3408}, 'weighted avg': {'precision': 0.7521379820865985, 'recall': 0.7414906103286385, 'f1-score': 0.7452790189390589, 'support': 3408}}


C:\Users\amaan\anaconda3\envs\t_mining\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [16]:
tup_linear_svc_standardised = make_pipeline(df_long, LinearSVC(random_state=42), PARAM_GRID, standardise=False)

C:\Users\amaan\anaconda3\envs\t_mining\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['&', "'", 'n', '‘', '’'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'0': {'precision': 0.8294172932330827, 'recall': 0.7731055628558914, 'f1-score': 0.8002720471548402, 'support': 2283}, '1': {'precision': 0.5953125, 'recall': 0.6773333333333333, 'f1-score': 0.6336798336798337, 'support': 1125}, 'accuracy': 0.7414906103286385, 'macro avg': {'precision': 0.7123648966165413, 'recall': 0.7252194480946124, 'f1-score': 0.716975940417337, 'support': 3408}, 'weighted avg': {'precision': 0.7521379820865985, 'recall': 0.7414906103286385, 'f1-score': 0.7452790189390589, 'support': 3408}}


C:\Users\amaan\anaconda3\envs\t_mining\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [75]:
print(len(tup_linear_svc[1]))
print(type(tup_linear_svc[0]))
print(tup_linear_svc[0].predict(['You are an absolute assohle. Hope your religion goes and dies!']))

3408
<class 'imblearn.pipeline.Pipeline'>
[1]


In [20]:
tup_svc_standardised = make_pipeline(df_long, SVC(random_state=42), PARAM_GRID)
tup_svc = make_pipeline(df_long, SVC(random_state=42), PARAM_GRID, standardise=False)

C:\Users\amaan\anaconda3\envs\t_mining\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['&', "'", 'n', '‘', '’'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'0': {'precision': 0.8353467561521253, 'recall': 0.8177836180464302, 'f1-score': 0.8264718902169101, 'support': 2283}, '1': {'precision': 0.6453537936913896, 'recall': 0.6728888888888889, 'f1-score': 0.6588337684943428, 'support': 1125}, 'accuracy': 0.7699530516431925, 'macro avg': {'precision': 0.7403502749217574, 'recall': 0.7453362534676595, 'f1-score': 0.7426528293556265, 'support': 3408}, 'weighted avg': {'precision': 0.7726290088609493, 'recall': 0.7699530516431925, 'f1-score': 0.7711336017961682, 'support': 3408}}


C:\Users\amaan\anaconda3\envs\t_mining\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['&', "'", 'n', '‘', '’'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'0': {'precision': 0.8353467561521253, 'recall': 0.8177836180464302, 'f1-score': 0.8264718902169101, 'support': 2283}, '1': {'precision': 0.6453537936913896, 'recall': 0.6728888888888889, 'f1-score': 0.6588337684943428, 'support': 1125}, 'accuracy': 0.7699530516431925, 'macro avg': {'precision': 0.7403502749217574, 'recall': 0.7453362534676595, 'f1-score': 0.7426528293556265, 'support': 3408}, 'weighted avg': {'precision': 0.7726290088609493, 'recall': 0.7699530516431925, 'f1-score': 0.7711336017961682, 'support': 3408}}


In [84]:
print(tup_svc[0][-1])
print(type(tup_svc[0][-1]))

SVC(kernel='sigmoid', random_state=42)
<class 'sklearn.svm._classes.SVC'>


In [12]:
#Neural Network Classifier 
from sklearn.neural_network import MLPClassifier


In [ ]:
tup_bots_MLP = bots(tuple_of_generators[0], tuple_of_generators[1], MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(10,30,10,5), random_state=42, batch_size=128, max_iter=1000))

In [22]:
#Support Vector Classifier without Gridsearch


#tup_bots_svc = bots(tuple_of_generators[0], tuple_of_generators[1], SVC())


{'0': {'precision': 0.7183863460046548, 'recall': 0.8231111111111111, 'f1-score': 0.767191383595692, 'support': 1125}, '1': {'precision': 0.7929240374609782, 'recall': 0.6773333333333333, 'f1-score': 0.7305848513902207, 'support': 1125}, 'accuracy': 0.7502222222222222, 'macro avg': {'precision': 0.7556551917328165, 'recall': 0.7502222222222222, 'f1-score': 0.7488881174929563, 'support': 2250}, 'weighted avg': {'precision': 0.7556551917328165, 'recall': 0.7502222222222222, 'f1-score': 0.7488881174929563, 'support': 2250}}
<class 'dict'>
{'0': {'precision': 0.7123893805309734, 'recall': 0.8586666666666667, 'f1-score': 0.7787182587666264, 'support': 1125}, '1': {'precision': 0.8221476510067114, 'recall': 0.6533333333333333, 'f1-score': 0.7280832095096582, 'support': 1125}, 'accuracy': 0.756, 'macro avg': {'precision': 0.7672685157688424, 'recall': 0.756, 'f1-score': 0.7534007341381423, 'support': 2250}, 'weighted avg': {'precision': 0.7672685157688424, 'recall': 0.756, 'f1-score': 0.75340

/Users/chengyiyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 1125}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1125}, 'accuracy': 0.5, 'macro avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2250}, 'weighted avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2250}}
<class 'dict'>
{'0': {'precision': 0.7105064247921391, 'recall': 0.8355555555555556, 'f1-score': 0.7679738562091503, 'support': 1125}, '1': {'precision': 0.8004314994606256, 'recall': 0.6595555555555556, 'f1-score': 0.7231968810916178, 'support': 1125}, 'accuracy': 0.7475555555555555, 'macro avg': {'precision': 0.7554689621263824, 'recall': 0.7475555555555555, 'f1-score': 0.745585368650384, 'support': 2250}, 'weighted avg': {'precision': 0.7554689621263824, 'recall': 0.7475555555555555, 'f1-score': 0.745585368650384, 'support': 2250}}
<class 'dict'>


In [17]:
#RandomForestClassifier without Gridsearch

from sklearn.ensemble import RandomForestClassifier


tup_bots_forest = bots(tuple_of_generators[0], tuple_of_generators[1], RandomForestClassifier(n_jobs=-1, random_state=42))

{'0': {'precision': 0.7456896551724138, 'recall': 0.7688888888888888, 'f1-score': 0.7571115973741794, 'support': 1125}, '1': {'precision': 0.7614678899082569, 'recall': 0.7377777777777778, 'f1-score': 0.7494356659142213, 'support': 1125}, 'accuracy': 0.7533333333333333, 'macro avg': {'precision': 0.7535787725403353, 'recall': 0.7533333333333333, 'f1-score': 0.7532736316442004, 'support': 2250}, 'weighted avg': {'precision': 0.7535787725403355, 'recall': 0.7533333333333333, 'f1-score': 0.7532736316442004, 'support': 2250}}
<class 'dict'>
{'0': {'precision': 0.73502722323049, 'recall': 0.72, 'f1-score': 0.7274360125729681, 'support': 1125}, '1': {'precision': 0.725609756097561, 'recall': 0.7404444444444445, 'f1-score': 0.7329520457545093, 'support': 1125}, 'accuracy': 0.7302222222222222, 'macro avg': {'precision': 0.7303184896640255, 'recall': 0.7302222222222222, 'f1-score': 0.7301940291637388, 'support': 2250}, 'weighted avg': {'precision': 0.7303184896640255, 'recall': 0.73022222222222

############################## LIME ###################
Need explainer object, instance of test data = tuple_of_generators[1][whichever dataset performed best], instance of the classifier rf = pull from list tup_bots_forst[0][whichever dataset peformed best]

In [41]:
from lime.lime_text import LimeTextExplainer
class_names = ['Hateful', 'Non-Hateful']
exp = LimeTextExplainer(class_names=class_names)

def best_val_set(tup, r):
    pass

def best_clf(tup, r):
    """Input tuple of (classifiers_generator, predictions_generator) and index 
    of classifier to extract from tuple -> outputs desired classifier"""
    lst_of_classifiers = [] # empty list for storing
    for i in range(0, r): # iteration for looping through sequence
        classifier = next(tup[0]) # access each classifier object
        lst_of_classifiers.append(classifier) #append it to empty list
    return lst_of_classifiers.pop() #pop out the final entry to the list - desired classifier

In [32]:
best_rf = best_clf(tup_bots_forest, 1) #best performing classifier
print(best_rf)

In [30]:
print(tup_bots_forest[0])

In [27]:
def hi(r):
    lst = [1, 2, 3, 4, 5, 6, 7]
    print(lst[0])
    new_lst = []
    for i in range(0, r):
        clf = lst.pop()
        new_lst.append(clf)
    return new_lst

In [28]:
print(hi(3))
print(hi(0))
print(hi(1))

1
[7, 6, 5]
1
[]
1
[7]


In [46]:
#classification report
from sklearn.metrics import accuracy_score,recall_score,precision_score,recall_score,f1_score
import warnings
warnings.filterwarnings('ignore')

def report(prediction, real_labels, data):
    print("classification report as follows: ")
    print(f'   Accuracy: {accuracy_score(prediction, real_labels)}')
    print(f'   Precision: {precision_score(prediction, real_labels,average="macro")}')
    print(f'   recall: {recall_score(prediction, real_labels,average="macro")}')
    print(f'   F1 measure: {f1_score(prediction, real_labels,average="macro")}')
    print('Show 5 example of correctly classified datapoint: ')
    if data[prediction==real_labels].shape[0] > 5:
        display(data[prediction==real_labels].iloc[:5,:])
    else:
        display(data[prediction==real_labels])
    print('Show 5 example of wrongly classified datapoint: ')
    if data[prediction!=real_labels].shape[0] > 5:
        display(data[prediction!=real_labels].iloc[:5,:])
    else:
        display(data[prediction!=real_labels])

In [132]:
val_x = vectorizer.transform(validation_set['Tokens'])
val_x = transformer.transform(val_x)

predict = ntwk.predict(val_x)
report(predict, validation_set['label'], validation_set[['Tokens','Label']])

classification report as follows: 
   Accuracy: 0.8168943476626144
   Precision: 0.7084592624109877
   recall: 0.7811589138333501
   F1 measure: 0.7309291098045785
Show 5 example of correctly classified datapoint: 


,Tokens,Label
23121,rt strategic vote kat food truly awful #mkr,0
35928,pancakes proof deity love us #mkr,0
45316,sick see fuck asshole bitch make chain latters...,1
42230,i'm try get insight trans issue definitely gro...,0
40128,make,0


Show 5 example of wrongly classified datapoint: 


,Tokens,Label
48658,lol ralph guy still moi era,1
30437,kill muslims oppose kill ezidis christians non...,0
28396,single men cannot adopt,1
44924,muslim brotherhood usa hundred years liken say...,0
7346,rt #mosul christian pastor #paul_jacob sentenc...,1
